#### From this site: https://github.com/jasonbaldridge/try-tf

In [ ]:
import tensorflow as tf
import pandas     as pd
import numpy      as np
import sys
import matplotlib.pyplot as plt
from   sklearn.utils           import shuffle
from   sklearn.model_selection import train_test_split

FEATURE_COUNT = 2
L1_SIZE       = 5           # How many nodes in layer 1
DROPOUT_KEEP  = 1.0
BATCH_SIZE    = 100
NUM_CLASSES   = 2
LR            = .01
EPOCHS        = 500
FILENM        = 'moons_train.csv'
SAVE_DIR      = "/home/tom/tf_checkpoints/"
SAVE_FILE     = "moons"
TB_DIR        = '/home/tom/tf_logs/'
PLOT_DATA     = False
VALID_PCT     = 0.15
EARLY_STOP    = 90
L1_BN         = True             # Batch Normalization

In [ ]:
# Init weights method. (Lifted from Delip Rao: http://deliprao.com/archives/100)
def init_weights(shape, init_method='xavier', xavier_params = (None, None)):
    if init_method == 'zeros':
        return tf.Variable(tf.zeros(shape, dtype=tf.float32))
    elif init_method == 'uniform':
        return tf.Variable(tf.random_normal(shape, stddev=0.01, dtype=tf.float32))
    else: #xavier
        (fan_in, fan_out) = xavier_params
        low = -4*np.sqrt(6.0/(fan_in + fan_out)) # {sigmoid:4, tanh:1} 
        high = 4*np.sqrt(6.0/(fan_in + fan_out))
        return tf.Variable(tf.random_uniform(shape, minval=low, maxval=high, dtype=tf.float32))

In [ ]:
if sys.platform[:3] =='win':
    data_loc = 'D:/Data/'
else:
    data_loc = "/home/tom/data/"

df = pd.read_csv(data_loc+FILENM, header=None)

In [ ]:
if PLOT_DATA:
    df0= df[df[0]==0]
    df1= df[df[0]==1]
    del(df0[0])
    del(df1[0])

    plt.figure(figsize=(5, 5))
    plt.scatter(df0[1], df0[2], color='r')
    plt.scatter(df1[1], df1[2])
    plt.show()

In [ ]:
# Both the data and the label are in the passed dataframe
# Split them apart and put into numpy array
def split_label(df):
    labels = np.array(df[0], dtype='f')
    # convert labels to a "one-hot" vector
    labels = (np.arange(NUM_CLASSES) == labels[:, None]).astype(np.float32)

    data = df.iloc[:,1:]
    # Turn "data" into a np array which will be used to load the tensor
    data = np.array(data, dtype='f')
    return data, labels

In [ ]:
df_train, df_val = train_test_split(df, test_size=VALID_PCT)
print('Training set:  {:,} rows'.format(df.shape[0]))

# Split the features from the labels
train_data, train_labels = split_label(df_train)
val_data,   val_labels   = split_label(df_val)

In [ ]:
tf.reset_default_graph()

image_batch = tf.placeholder(tf.float32, shape=[None, FEATURE_COUNT], name="image")
label_batch = tf.placeholder(tf.float32, shape=[None, FEATURE_COUNT], name="label")
dropout     = tf.placeholder(tf.float32, name="drop")

l0_w = init_weights(
    [FEATURE_COUNT, L1_SIZE],
    'xavier',
    xavier_params=(FEATURE_COUNT, L1_SIZE))
l0_b = init_weights([1,L1_SIZE],'zeros')

In [ ]:
l0_out = tf.nn.tanh(tf.matmul(image_batch,l0_w + l0_b), name="l0")
l0_out = tf.nn.dropout(l0_out, keep_prob=dropout)
if L1_BN:
    l0_out = tf.contrib.layers.batch_norm(l0_out)

l1_w = init_weights(
    [L1_SIZE, NUM_CLASSES],
    'xavier',
    xavier_params=(L1_SIZE, NUM_CLASSES))
l1_b = init_weights([1,NUM_CLASSES],'zeros')
y    = tf.nn.softmax(tf.matmul(l0_out, l1_w) + l1_b, name="y")

In [ ]:
# Optimization
cross_entropy = -tf.reduce_sum(label_batch*tf.log(y))
optimize      = tf.train.AdamOptimizer(LR, name="optimize").minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(label_batch,1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, "float"), name="accuracy")

In [ ]:
# Capture Tensorboard data
summary_loss     = tf.scalar_summary('Loss function', cross_entropy)
summary_accuracy = tf.scalar_summary('Validation', accuracy)
count = 0

In [ ]:
num_training_batches = int(len(train_data)/BATCH_SIZE)
stop_count = 0
best_score = 0.

saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.initialize_all_variables())
train_writer = tf.train.SummaryWriter(TB_DIR + 'run4', sess.graph)
for i in range(EPOCHS):
    x,y = shuffle(train_data,train_labels)
    for j in range(num_training_batches):
            mini_x  = x[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
            mini_y = y[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
            _, TB = sess.run([optimize, summary_loss], feed_dict={image_batch:mini_x,
                                                                  label_batch:mini_y,
                                                                  dropout:DROPOUT_KEEP})
            if j % 5 == 0:
                count += 1
                train_writer.add_summary(TB, count)
    score, TB = sess.run([accuracy, summary_accuracy], feed_dict={image_batch:val_data,
                                                                  label_batch:val_labels,
                                                                  dropout:1.0})
    train_writer.add_summary(TB, i)
    if score > best_score:
        best_score = score
    else:
        stop_count += 1
    if stop_count > EARLY_STOP:
        print("Stopping early")
        break

saver.save(sess, SAVE_DIR+SAVE_FILE)
train_writer.close()
sess.close()
!aplay /usr/share/sounds/bicycle_bell.wav